In [ ]:
import bibliograph as bg

bibtex_fname = "bibliograph/test_data/bibtex_test_data_short.bib"
entry_syntax_fname = "bibliograph/resources/default_bibtex_syntax.csv"

tn = bg.slurp_bibtex(
    bibtex_fname,
    entry_syntax_fname,
    syntax_case_sensitive=False,
    space_char='|',
    na_string_values='!',
    na_node_type='missing'
)

tn.strings

In [ ]:
    import bibliograph as bg

    tn = bg.slurp_shorthand(
        'bibliograph/test_data/manual_annotation.shnd',
        "bibliograph/resources/default_entry_syntax.csv",
        link_syntax_fname="bibliograph/resources/default_link_syntax.csv",
        syntax_case_sensitive=False,
        item_separator='__',
        default_entry_prefix='wrk',
        space_char='|',
        na_string_values='!',
        na_node_type='missing',
        skiprows=2,
        comment_char='#'
    )

    notna_subset = ['node_id', 'string', 'date_inserted']

    assert not tn.strings[notna_subset].isna().any(axis=None)

In [4]:
    import bibliograph as bg


    tn = bg.slurp_shorthand(
        'bibliograph/test_data/manual_annotation.shnd',
        "bibliograph/resources/default_entry_syntax.csv",
        link_syntax_fname="bibliograph/resources/default_link_syntax.csv",
        syntax_case_sensitive=False,
        item_separator='__',
        default_entry_prefix='wrk',
        space_char='|',
        na_string_values='!',
        na_node_type='missing',
        skiprows=2,
        comment_char='#'
    )

    shorthand_text_string = tn.resolve_strings().query(
        'node_type == "shorthand_text"'
    )
    shorthand_text_string.string.squeeze()


'asmith'

# TODO: alias insertion executes without error, now write test cases

In [ ]:
!pytest bibliograph/tests.py

In [ ]:
import pandas as pd
strings = pd.DataFrame({'string':['a','A','b','c','d','C','z']})
data = pd.Series([1,1,2,4,5,6,3,3,2])

data = pd.concat([data, data.map(strings.string)], axis='columns')
data[2] = data[1].str.casefold()
folded_uid = data[2].unique()
folded_uid = pd.Series(range(len(folded_uid)), index=folded_uid)
data[3] = data[2].map(folded_uid)
data

In [19]:
import bibliograph as bg
import shorthand as shnd

def slurp_shorthand(
    shorthand_fname,
    entry_syntax_fname,
    link_syntax_fname=None,
    syntax_case_sensitive=True,
    allow_redundant_items=False,
    aliases_dict=None,
    aliases_case_sensitive=True,
    **kwargs
):

    s = shnd.Shorthand(
        entry_syntax=entry_syntax_fname,
        link_syntax=link_syntax_fname,
        syntax_case_sensitive=syntax_case_sensitive,
        allow_redundant_items=allow_redundant_items
    )

    parsed = s.parse_text(shorthand_fname, **kwargs)

    tn = bg.core.build_textnet_assertions(
        parsed,
        shorthand_fname,
        'file_name'
    )

    tn = bg.core.complete_textnet_from_assertions(
        tn,
        parsed,
        aliases_case_sensitive
    )

    return tn

tn = slurp_shorthand(
    'bibliograph/test_data/manual_annotation.shnd',
    "bibliograph/resources/default_entry_syntax.csv",
    link_syntax_fname="bibliograph/resources/default_link_syntax.csv",
    syntax_case_sensitive=False,
    item_separator='__',
    default_entry_prefix='wrk',
    space_char='|',
    na_string_values='!',
    na_node_type='missing',
    skiprows=2,
    comment_char='#'
)

#tn.strings.query('node_type_id == 0')
tn.id_lookup('node_types', 'shorthand_text')
tn.nodes.query('node_type_id == 0')
tn.strings.loc[60]

resolved = tn.strings.copy()
resolved['node_type'] = resolved['node_id'].map(tn.nodes['node_type_id'])
#resolved['node_type'] = resolved['node_id'].map(tn.node_types['node_type'])
resolved

tn.resolve_strings()

,node_id,string,date_inserted,date_modified,node_type
0,0,asmith,2022-07-22 16:06:31,<NA>,4
1,1,bwu,2022-07-22 16:06:31,<NA>,4
2,2,1999,2022-07-22 16:06:31,<NA>,5
3,3,101,2022-07-22 16:06:31,<NA>,6
4,4,803,2022-07-22 16:06:31,<NA>,6
...,...,...,...,...,...
60,60,This is stuff shorthand ignores when you use s...,2022-07-22 16:06:31,<NA>,0
61,61,Shorthand.parse_text,2022-07-22 16:06:31,<NA>,3
62,62,"default_entry_prefix,item_separator,entry_pref...",2022-07-22 16:06:31,<NA>,1
63,63,"left_entry_prefix,right_entry_prefix,source_po...",2022-07-22 16:06:31,<NA>,2
